In [2]:
import torch 
import pandas as pd
import torch.nn as nn
import time
import io
import joblib
import argparse

from torchsummary import summary
from tqdm import tqdm
from tabulate import tabulate
from sklearn.preprocessing import *

from preprocessor import Preprocessor
from AutoCleanse.utils import *
from AutoCleanse.dataloader import PlainDataset, DataLoader
from AutoCleanse.autoencoder import *
from AutoCleanse.loss_model import loss_CEMSE
# from AutoCleanse.preprocessor import Preprocessor
from AutoCleanse.anonymize import anonymize
from AutoCleanse.bucketfs_client import *

ImportError: cannot import name 'bucketfs_client' from 'bucketfs_client' (c:\Users\zpakz\Downloads\AutoCleanse\AutoCleanse\AutoCleanse\bucketfs_client.py)

Device configuration

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)

## Setup directory path

In [3]:
import os
PROJECT_DIR = os.getcwd()
os.chdir(PROJECT_DIR)
DATASET_DIR = os.path.join(PROJECT_DIR,'dataset')
EVAL_DIR = os.path.join(PROJECT_DIR,'evaluate')

## Preparing data

Load dataframe and group features by their type

In [4]:
df = pd.read_csv(os.path.join(DATASET_DIR,'adult.csv')).drop(columns=['fnlwgt','income'])
continous_columns = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_columns = df.select_dtypes(include=['object', 'bool']).columns.tolist()
og_columns = df.columns.to_list()
df = df[continous_columns+categorical_columns]

Data preprocessing

In [5]:
scaler = MinMaxScaler()
onehotencoder = OneHotEncoder(handle_unknown='ignore',sparse_output=False)
preprocessor = Preprocessor(scaler,
                            onehotencoder,
                            continous_columns,
                            categorical_columns)

X_train,X_val,X_test = preprocessor.split(df=df,
                                        train_ratio=0.7,
                                        val_ratio=0.15,
                                        test_ratio=0.15,
                                        random_seed=42)
X_dirty = replace_with_nan(X_test,0.0,42)

X_train = preprocessor.fit_transform(input_df=X_train)

X_val = preprocessor.transform(input_df=X_val)                          

X_test = preprocessor.transform(input_df=X_test)  

X_dirty = preprocessor.transform(input_df=X_dirty)
categories = preprocessor.encoder.categories_

/home/zahra/miniforge3/envs/autocleanse/lib/python3.10/site-packages/AutoCleanse/utils.py:99: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[48, 28, 38, nan, nan, 27, 25, 20, 39, 52, 40, 35, 55, 62, nan, 17, 22, nan, 17, 51, 20, nan, 44, 19, 21, 77, 46, nan, 21, 48, 39, 34, nan, nan, 32, 52, 30, 20, 46, 30, 20, 41, 39, 21, 47, nan, 28, nan, nan, 21, 23, 24, 35, nan, nan, 49, 65, 42, nan, 21, nan, 23, 46, 22, 44, 30, 29, 56, nan, 40, 32, 25, nan, nan, nan, 42, 25, 39, 28, 39, 31, nan, 59, 28, 48, 38, nan, 20, 46, 38, 64, nan, 44, 29, 48, 47, 46, 26, nan, 47, 18, 50, nan, 28, 23, 33, 18, 39, 30, 18, 36, 37, 23, 36, 19, 34, nan, 42, nan, 48, 33, 49, 24, nan, 27, 53, nan, 43, 42, 66, 32, nan, 36, 27, 37, 36, 29, 26, 23, 37, 74, 42, 29, nan, 27, 21, 41, 60, 38, 62, 37, 55, 67, 48, 53, 23, 59, 37, 34, 37, 48, 25, 47, 49, nan, 30, 28, 44, 22, 30, nan, 18, 51, 30, 28, 32, 51, 33, 39, 68, 24, 52, 59, 33, 55, 27, nan, 31, 23

In [ ]:
X_dirty

(Optional) Check save/load function of preprocessor

Both functiona take in 2 parameters:

    - Suffix of the preprocessor name, in the example below would be **preprocessor_main.pkl**
    - Save/load location: can either be "local" to save/load in the home folder or "bucketfs" to save/load to/from Exasol BucketFS

In [ ]:
preprocessor.save("main","local")
preprocessor.save("main","bucketfs",url="http://172.18.0.2:6583",bucket="default",user="w",password="write")
preprocessor2 = Preprocessor(scaler=MinMaxScaler(),encoder=OneHotEncoder(sparse=False))
preprocessor2.load("main","local")
preprocessor2.load("main","bucketfs",url="http://172.18.0.2:6583",bucket="default",user="w",password="write")

Convert dataframes into datasets, and create dataloaders

In [6]:
batch_size = 64

In [7]:
train_dataset = PlainDataset(X_train)
val_dataset = PlainDataset(X_val)
test_dataset = PlainDataset(X_test)
dirty_dataset = PlainDataset(X_dirty)

def custom_collate_fn(batch):
    tensor_data = torch.stack([item[0] for item in batch])
    indices = [item[1] for item in batch]
    return tensor_data, indices

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True,collate_fn=custom_collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=True, collate_fn=custom_collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=True, collate_fn=custom_collate_fn)
dirty_loader = DataLoader(dirty_dataset, batch_size=batch_size, shuffle=False, drop_last=True, collate_fn=custom_collate_fn)

## Instantiate model

In [8]:
layers = [X_train.shape[1],1024,128]   
wlc = (1,1) 

autoencoder = Autoencoder(layers=layers,dropout_enc=[(0,0.0)],dropout_dec=[(0,0.1)], batch_norm=True, \
                          learning_rate=1e-4,weight_decay=1e-5,l1_strength=1e-5,l2_strength=1e-5)

In [9]:
summary(autoencoder.to(device),torch.tensor(X_train.values).float().to(device).shape[1:])

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 1024]         110,592
       BatchNorm1d-2                 [-1, 1024]           2,048
              ReLU-3                 [-1, 1024]               0
           Dropout-4                 [-1, 1024]               0
            Linear-5                  [-1, 128]         131,200
       BatchNorm1d-6                  [-1, 128]             256
              ReLU-7                  [-1, 128]               0
            Linear-8                 [-1, 1024]         132,096
              ReLU-9                 [-1, 1024]               0
          Dropout-10                 [-1, 1024]               0
           Linear-11                  [-1, 107]         109,675
             ReLU-12                  [-1, 107]               0
           Linear-13                  [-1, 107]          11,556
Total params: 497,423
Trainable params:

(Optional) Model can be loaded from checkpoint after instantiation

The function takes in 2 parameters:

    - Suffix of the preprocessor name, in the example below would be **autoencoder_main.pkl**
    - Save/load location: can either be "local" to load in the home folder or "bucketfs" to load from Exasol BucketFS

In [9]:
autoencoder.load("bucketfs","main",url="http://172.18.0.2:6583",bucket="default",user="w",password="write")

Loaded weight from default/autoencoder/autoencoder_main.pth


In [10]:
autoencoder.load("local","main")

Loaded weight from autoencoder_main.pth


### Train the model

In [ ]:
autoencoder.train_model(
      patience=10,
      num_epochs=1,
      batch_size=batch_size,
      train_loader=train_loader,
      val_loader=val_loader,
      continous_columns=continous_columns, 
      categorical_columns=categorical_columns, 
      categories=categories,
      device=device,
      wlc=wlc)

(Optional) Model can be saved after training

The function takes in 2 parameters:

    - Suffix of the preprocessor name, in the example below would be **autoencoder_main.pkl**
    - Save/load location: can either be "local" to load in the home folder or "bucketfs" to load from Exasol BucketFS

In [ ]:
autoencoder.save("bucketfs","main",url="http://172.18.0.2:6583",bucket="default",user="w",password="write")

### Use trained model to clean data

In [10]:
cleaned_data = autoencoder.clean(dirty_loader=dirty_loader,
                                test_loader=test_loader,
                                df=X_dirty,
                                batch_size=batch_size,
                                continous_columns=continous_columns, 
                                categorical_columns=categorical_columns, 
                                og_columns=og_columns,
                                onehotencoder=preprocessor.encoder, 
                                scaler=preprocessor.scaler,
                                device=device) 

Clean progress:   0%|          | 0/76 [00:00<?, ?it/s]

Clean progress: 100%|██████████| 76/76 [00:04<00:00, 15.82it/s]


MAE: 0.00116814

MSE: 0.00012697


In [ ]:
# original data
print(tabulate(df.loc[[28296,28217,8054,4223,22723],og_columns],headers=og_columns,tablefmt="simple",maxcolwidths=[None, 4]))

In [ ]:
# cleaned data
print(tabulate(cleaned_data.loc[[28296,28217,8054,4223,22723]],headers=cleaned_data.columns.to_list(),tablefmt="simple",maxcolwidths=[None, 4]))

### Use trained model to anonymize data

In [11]:
anonymized_data = autoencoder.anonymize(df=X_test,
                                        data_loader=test_loader,
                                        batch_size=batch_size,
                                        device=device)

Anonymize progress: 100%|██████████| 76/76 [00:02<00:00, 31.47it/s]


In [ ]:
# anonymized data
print(tabulate(anonymized_data.round(decimals=4).iloc[:5,:32],headers=anonymized_data.columns.to_list(),tablefmt="simple",maxcolwidths=[None, 6]))